In [4]:
import pandas as pd
import numpy as np
import re
import spacy
from nltk.tokenize import sent_tokenize
#nltk.download('punkt')

In [5]:
df = pd.read_csv('data.csv', index_col=0)

In [151]:
df.head()

## Spacy

CCONJ??

keep punctuation: The service is fast, room is clean

In [8]:
nlp = spacy.load("en_core_web_sm")

In [160]:
class NounAndAdjPair:
    
    patterns=['(ADJ )*(NOUN )+(ADV )*(VERB )+(ADV )*ADJ ', # The Korean grill is good
              '(NOUN )+.*PRON (VERB )+(ADV )*ADJ ', 
              '(ADV )*(ADJ )+(NOUN )+', # good service
             ]
    
    def __init__(self, doc):
        
        def getDoc():
            return re.sub(' +', ' ', self.original_doc.lower().replace('\n', ' ').strip()).lower()#.replace('soooo', 'so') 6

        def getSentences():
            sentences = [s for s in sent_tokenize(self.doc)] # nltk sentence tokenizer
            return [' '.join([w.text for w in nlp(s)]) for s in sentences] # spacy word tokenizer

        def getTaggings():
            return [' '.join([w.pos_ for w in nlp(s)])+' ' for s in self.sentences]  
        
        self.original_doc = doc
        self.doc = getDoc()
        self.sentences = getSentences()
        self.taggings = getTaggings()
#         print(len(self.sentences), len(self.taggings))

    def getPairsWithFSA(self, returnInDf=True):
        
        def checkIfAdjAndNounExists(tagging):
            if 'ADJ' in tagging and 'NOUN' in tagging:
                return True
            return False
        
        def getWoldsByTaggingIndex(target_tagging_index, no_sentence, no_pattern):
            
            original_tagging = self.taggings[no_sentence]
            
            baseIndex = 0 if target_tagging_index[0]==0 else len(original_tagging[:target_tagging_index[0]].strip().split(' '))
            buildIndex = len(original_tagging[target_tagging_index[0]:target_tagging_index[1]].strip().split(' '))
            
            trimmed_tagging_list = original_tagging.strip().split(' ')[baseIndex: baseIndex+buildIndex]
            trimmed_sentence_list = self.sentences[no_sentence].split(' ')[baseIndex: baseIndex+buildIndex]
            
            
            noun_adj_pair = []
            
#             print(trimmed_sentence_list, '\n', trimmed_tagging_list)
            
            if no_pattern==0: # the first pattern
                last_verb_index = len(trimmed_tagging_list)-trimmed_tagging_list[::-1].index('VERB')-1
                for i in range(len(trimmed_tagging_list)):
                    if trimmed_tagging_list[i] not in ['ADJ', 'NOUN']: #filter out adv
                        break
                        
                noun_adj_pair.append(' '.join(trimmed_sentence_list[:i]))
                noun_adj_pair.append(' '.join(trimmed_sentence_list[last_verb_index+1:]))
                
            elif no_pattern==1:
                last_noun_end_index = len(trimmed_tagging_list)-trimmed_tagging_list[::-1].index('NOUN')-1
                for i in range(last_noun_end_index, 0, -1):
                    if trimmed_tagging_list[i]=='NOUN':
                        last_noun_start_index = i
                    else:
                        break
                
                last_verb_index = len(trimmed_tagging_list)-trimmed_tagging_list[::-1].index('VERB')-1
                        
                noun_adj_pair.append(' '.join(trimmed_sentence_list[last_noun_start_index:last_noun_end_index+1]))
                noun_adj_pair.append(' '.join(trimmed_sentence_list[last_verb_index+1:]))
            
            elif no_pattern==2:
                noun_index = trimmed_tagging_list.index('NOUN')
                noun_adj_pair.append(' '.join(trimmed_sentence_list[noun_index:]))
                noun_adj_pair.append(' '.join(trimmed_sentence_list[:noun_index]))
                
            
            return tuple(noun_adj_pair)
        
        
        def formDf(sentence_adjNoun_pair):
            df = pd.DataFrame(columns=['Sentence', 'AdjNounPair'])
            pd.set_option('display.max_colwidth', -1)
            df.Sentence, df.AdjNounPair = list(sentence_adjNoun_pair.keys()), list(sentence_adjNoun_pair.values())
            return df
        
        sentence_adjNoun_pair = dict(zip(self.sentences, [[] for i in range(len(self.sentences))]))
        
        for i in range(len(self.sentences)):
            s = self.sentences[i]
            t = self.taggings[i]
            
            if checkIfAdjAndNounExists(t):
                for p in self.patterns:
                    for x in re.finditer(p, t):
                        sentence_adjNoun_pair[s].append(getWoldsByTaggingIndex(x.span(), i, self.patterns.index(p)))
                        
                        
        if returnInDf:
            return formDf(sentence_adjNoun_pair)
            
        return sentence_adjNoun_pair



In [162]:
a = NounAndAdjPair(df.text[2])
a.getPairsWithFSA()